In [72]:
# from .dataset_scripts import *
import matplotlib.pyplot as plt
from icepyx import Query
import os
from icepyx import core
import h5py
import readers as rd

In [7]:
short_name = 'ATL03' 
# '''fill in with name of whichever DataSet this is a member function of'''

bbox = [-140,68,-120,80]
timeframe = ['2019-06-10','2019-06-16']
earthdata_uid = 'mwiering'
email = 'molly.m.wieringa@gmail.com'

data = Query(short_name, bbox, timeframe)
data.avail_granules()
    
print(data.avail_granules())

if ('y' or 'yes' == input("Here's what we've got! Continue? (y/n)")):
    data.earthdata_login(earthdata_uid, email)
        
    path = './colocated/is2/' + short_name
    data.download_granules(path)
    files = data.avail_granules(ids=True)
        
    print("Downloaded! Have a great day!")
else:
    print("Nothing was downloaded")
    exit()

{'Number of available granules': 36, 'Average size of granules (MB)': 1754.6772770881612, 'Total size of all granules (MB)': 63168.38197517381}


Earthdata Login password:  ············


Total number of data order requests is  4  for  36  granules.
Data request  1  of  4  is submitting to NSIDC
order ID:  5000000993432
Initial status of your order request at NSIDC is:  pending
Your order status is still  pending  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue wa

In [73]:
thing ={}
for file in files[0]:
    f = h5py.File(path + '/processed_' + file, 'r')
    orientation  = f['orbit_info/sc_orient'][0] # Check forward/backward orientation
    beam = None
    if orientation == 0:
        # take left beam if backward orientation
        beam = 'gt1l'
    elif orientation == 1:
        # take right beam if in forward orientation
        beam = 'gt1r'
    else:
        print('Error: Beam orientation unknown. Process terminated.')
    
    df = rd.getATL03(file,beam)
        
    #trim to bounding box
    df_cut=df[bbox]
        
    #convert time to UTC
    epoch=f['/ancillary_data/atlas_sdp_gps_epoch'][0]
    df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime
        
    #caculate along track distance
    df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]
        
    #save a dictionary entry in the DataSet
    thing[file] = df_cut
        
        

TypeError: string indices must be integers

In [92]:
f[beam].keys()

<KeysViewHDF5 ['bckgrd_atlas', 'signal_find_output']>